In [ ]:
import boto3
import re 

In [ ]:
bucket_name = "ec2-east-sagemaker-demo" 

train_file = r"Logistic/Train/train.csv"
test_file = r"Logistic/Test/test.csv"
val_file = r"Logistic/Val/val.csv"

model_folder = r"Logistic/model/"

In [ ]:
s3model_output = r"s3://{0}/{1}".format(bucket_name, model_folder) 

s3_train = r"s3://{0}/{1}".format(bucket_name, train_file) 

s3_test = r"s3://{0}/{1}".format(bucket_name, test_file) 

s3_val = r"s3://{0}/{1}".format(bucket_name, val_file) 

In [ ]:
s3model_output

In [ ]:
with open("train.csv", "rb") as f: 

boto3.Session().resource("s3").Bucket(bucket_name).Object(train_file).upload_fileobj(f) 

 

with open("test.csv", "rb") as f: 

boto3.Session().resource("s3").Bucket(bucket_name).Object(test_file).upload_fileobj(f) 

# Build and Train Model in Sagemaker 

In [ ]:
import sagemaker 

from sagemaker import get_execution_role

In [ ]:
sagemakerSess = sagemaker.Session() 

role = get_execution_role()

In [ ]:
sagemakerSess.boto_region_name 

In [ ]:
ECRdockercontainer = sagemaker.amazon.amazon_estimator.get_image_uri(
    sagemakerSess.boto_region_name, "linear-learner", "latest"
    )

In [ ]:
logistic_model = sagemaker.estimator.Estimator(
    image_name = ECRdockercontainer, 
    role = role, 
    train_instance_count = 1, 
    train_instance_type = "m3.t1.medium", 
    output_path = s3model_output, 
    sagemaker_session = sagemakerSess, 
    base_job_name = "Logistic-Demo-v1"
    )

In [ ]:
logistic_model.set_hyperparameters(
    predictor_type = "binary_classifier", 
    mini_batch_size = 100
    ) 
logistic_model.hyperparameters()

In [ ]:
train_config = sagemaker.session.s3_imput(s3_data = s3_train, contenct_type = "text/csv")

In [ ]:
logistic_model.fit({"train": train_config}) 